Import the libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
import tensorflow as tf
from google.colab import files

load and prepare the dataset

In [ ]:
file_path = "clickbait.csv"  # Path to your CSV file
data = pd.read_csv(file_path)

# Display the first few rows to confirm the data structure
print(data.head())

                                               title  label
0  15 Highly Important Questions About Adulthood,...      1
1  250 Nuns Just Cycled All The Way From Kathmand...      1
2  Australian comedians "could have been shot" du...      0
3  Lycos launches screensaver to increase spammer...      0
4  Fußball-Bundesliga 2008–09: Goalkeeper Butt si...      0


preprocess the text and the labels

In [ ]:
titles = data['title'].values
labels = data['label'].values

Tockenize and pad the sequences

In [ ]:
vocab_size = 10000  # Maximum number of words in the vocabulary
max_len = 50  # Maximum number of words in a title (padding/truncating length)

# Initialize and fit the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(titles)

#convert the text to sequence
sequences = tokenizer.texts_to_sequences(titles)

# pad the sequences to the same length
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

Split the dataset into training and testing
and validation


In [ ]:
# # spliting data for traingingNtesting and validation
# X_trainNtest, X_validate, Y_trainNtest, Y_validate = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# # spliting data for training and testing
# X_train, X_test, Y_train, Y_test = train_test_split(X_trainNtest, Y_trainNtest, test_size=0.2, random_state=42)

# First, split into training + validation and test sets
X_trainNval, X_test, Y_trainNval, Y_test = train_test_split(padded_sequences, labels, test_size=0.6, random_state=42)

# Then, split the training + validation set into training and validation sets
X_train, X_validate, Y_train, Y_validate = train_test_split(X_trainNval, Y_trainNval, test_size=0.4, random_state=32)


# random number that is used when shuffling data.
# if this is not given it will generate a random number and use it to shuffle. but then if we do this again we'll get a different split. but if we configure once like this and also the next time
# then the same split can be acquried.

creating the RNN

In [ ]:
embedding_dim = 128  # Embedding vector size

# initial_model = Sequential([
#     Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
#     Conv1D(filters=128, kernel_size=5, activation='relu'),
#     GlobalMaxPooling1D(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])


initial_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.6),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Compile the model

In [ ]:
initial_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(len(padded_sequences))
print(len(labels))

print(len(X_trainNval))
print(len(X_validate))
print(len(X_train))
print(len(X_test))



31986
31986
12794
5118
7676
19192


Train the model

In [ ]:
history = initial_model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_data=(X_validate, Y_validate))



Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.7822 - loss: 1.0028 - val_accuracy: 0.9590 - val_loss: 0.4750
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9844 - loss: 0.1860 - val_accuracy: 0.9615 - val_loss: 0.1659
Epoch 3/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9959 - loss: 0.0607 - val_accuracy: 0.9623 - val_loss: 0.1138
Epoch 4/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9975 - loss: 0.0269 - val_accuracy: 0.9613 - val_loss: 0.1244
Epoch 5/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9986 - loss: 0.0175 - val_accuracy: 0.9610 - val_loss: 0.1337


Additionally generated Data prepartion for validation

In [ ]:
file_path = "generated_clickbait.csv"  # Path to your CSV file
data = pd.read_csv(file_path)

# Display the first few rows to confirm the data structure
print(data.head())


generated_title = data['title'].values
generated_labels = data['label'].values

print(len(generated_title))
print(len(generated_labels))

#convert the text to sequence
generated_validation_sequences = tokenizer.texts_to_sequences(generated_title)

# pad the sequences to the same length
padded_generated_validation_sequences = pad_sequences(generated_validation_sequences, maxlen=max_len, padding='post', truncating='post')

                                               title  label
0  21 Things Only People Who Love Coffee Will Und...      1
1  30 Things You Didn't Know You Needed Until Rig...      1
2       12 Ways To Make Your Mornings Less Miserable      1
3  Scientists Discover New Species In The Depths ...      0
4  Local Bakery Wins National Award For Best Croi...      0
126
126


Additional dataset downloaded from Kaggle, data preparation

In [ ]:
file_path = "kaggle_clickbait_data.csv"  # Path to your CSV file
data = pd.read_csv(file_path)

# Display the first few rows to confirm the data structure
print(data.head())


kaggle_title = data['title'].values
kaggle_labels = data['label'].values

print(len(generated_title))
print(len(generated_labels))

#convert the text to sequence
kaggle_test_sequences = tokenizer.texts_to_sequences(generated_title)

# pad the sequences to the same length
padded_kaggle_test_sequences = pad_sequences(kaggle_test_sequences, maxlen=max_len, padding='post', truncating='post')

                                               title  label
0                                 Should I Get Bings      1
1      Which TV Female Friend Group Do You Belong In      1
2  The New "Star Wars: The Force Awakens" Trailer...      1
3  This Vine Of New York On "Celebrity Big Brothe...      1
4  A Couple Did A Stunning Photo Shoot With Their...      1
32000
32000


Evaluate the model

In [ ]:
test_loss, test_acc = initial_model.evaluate(padded_kaggle_test_sequences, kaggle_labels)
print(f"Test Accuracy: {test_acc}")

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9639 - loss: 0.1261
Test Accuracy: 0.9696875214576721


perform predictions

In [ ]:
predictions = initial_model.predict(X_test)
print(predictions[:30])  # Display the first 10 predictions

# Convert probabilities to binary predictions
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

print(binary_predictions[:10])  # Display the first 10 binary predictions

200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[[1.0000000e+00]
 [1.5503602e-04]
 [1.0000000e+00]
 [3.0969048e-08]
 [1.0000000e+00]
 [9.9994171e-01]
 [9.9999976e-01]
 [7.4523103e-11]
 [1.6532070e-03]
 [9.9992537e-01]
 [1.0214184e-04]
 [9.9985981e-01]
 [1.5888825e-06]
 [1.0000000e+00]
 [3.4886951e-05]
 [5.5405515e-04]
 [1.9450525e-07]
 [5.9666822e-04]
 [9.9994969e-01]
 [9.9999976e-01]
 [1.0000000e+00]
 [2.4595835e-07]
 [1.0000000e+00]
 [9.9999595e-01]
 [1.0000000e+00]
 [1.4334894e-06]
 [1.0000000e+00]
 [3.7126327e-05]
 [9.9999821e-01]
 [2.0115014e-05]]
[[1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]]


Read the tartgetTitles from the train.jsonl and validation.jsonl

In [ ]:
import json

# Function to read JSONL files and extract targetTitles
def extract_titles_from_jsonl(file_path):
    titles = []
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            if 'targetTitle' in data:
                titles.append(data['targetTitle'])
    return titles

train_file_path = 'train.jsonl'
validation_file_path = 'validation.jsonl'

train_titles = extract_titles_from_jsonl(train_file_path)
validation_titles = extract_titles_from_jsonl(validation_file_path)

all_unlabelled_titles = train_titles + validation_titles


In [ ]:
print("First 10 training titles:")
for i, title in enumerate(train_titles[:10]):
    print(f"{i+1}. {title}")
print(len(train_titles))

print("First 10 validation titles:")
for i, title in enumerate(validation_titles[:10]):
    print(f"{i+1}. {title}")
print(len(validation_titles))

First 10 training titles:
1. Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better Idea
2. Hole In Ozone Layer Expected To Make Full Recovery By 2070: NASA
3. Intellectual Stimulation Trumps Money For Employee Happiness, Survey Finds
4. ‘Follow your passion’ is wrong, here are 7 habits you need instead
5. Revealed: The perfect way to cook rice so that it's perfectly fluffy and NEVER sticks to the bottom of the pan 
6. Here's what happens if your Apple AirPods get lost or stolen
7. The Reason Why Gabor Kiraly Wears THOSE Trackie Bottoms
8. You’ll Never Believe What This Family Saw in the Sky Outside Their House in Finland.
9. Should I Drink Red Wine?
10. Taste test: 29 hot sauce bottles, ranked
3200
First 10 validation titles:
1. Five Nights at Freddy’s Sequel Delayed for Weird Reason
2. Why Arizona Sheriff Joe Arpaio’s fate could hang on a single word
3. Here’s how much you should be tipping your hairdresser
4. Alan Rickman & Rupert Grint On 'CBGB,' Reuniting Post-'Pott

extracting padded seuqnces of unlabelled training data and validation data


In [ ]:
# function to extract padded sequences
def extract_padded_seuqnces(titles):

  unlabelled_sequences = tokenizer.texts_to_sequences(titles)
  padded_sequences = pad_sequences(unlabelled_sequences, maxlen=max_len, padding='post', truncating='post')
  return padded_sequences

padded_unlabelled_training_sequences = extract_padded_seuqnces(train_titles)
padded_unlabelled_validation_sequences = extract_padded_seuqnces(validation_titles)

padded_unlabelled_all_sequences = extract_padded_seuqnces(all_unlabelled_titles)


generate the pseudo labels for the unlabelled data

In [ ]:
# function to generate pseudo labels for unlabelled data and process them

def generate_formatted_pseudo_labels(padded_sequences):
  pseudo_labels = initial_model.predict(padded_sequences)
  pseudo_labels = (pseudo_labels > 0.5).astype(int).flatten() # rounding off the predicted values to 1 or 0.
  return pseudo_labels

# pseudo_labels_training = generate_formatted_pseudo_labels(padded_unlabelled_training_sequences)
# pseudo_labels_validation = generate_formatted_pseudo_labels(padded_unlabelled_validation_sequences)

pseudo_labels = generate_formatted_pseudo_labels(padded_unlabelled_all_sequences)

print(pseudo_labels)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[1 0 0 ... 1 0 0]


combine the

1.   labelled and unlabelled sequences
2.   labels and pseudo labels




In [ ]:
X_combined = np.vstack((padded_sequences, padded_unlabelled_all_sequences))
y_combined = np.concatenate((labels, pseudo_labels))

Split the unlabelled data set


In [ ]:
# final_X_train, final_X_test,  final_Y_train, final_Y_test = train_test_split(X_combined, y_combined, test_size=0.25, random_state=42)

# spliting data for trainingNtesting and validation
final_X_trainNval, final_X_test,  final_Y_trainNval, final_Y_test = train_test_split(X_combined, y_combined, test_size=0.25, random_state=42)

# spliting data for training and testing
final_X_train, final_X_validate,  final_Y_train, final_Y_validate = train_test_split(final_X_trainNval, final_Y_trainNval, test_size=0.3, random_state=42)



In [ ]:
print(len(final_X_trainNtest))
print(len(final_X_validate))
print(len(final_X_train))
print(len(final_X_test))

26989
8997
18892
8097


Build the final model

In [ ]:
final_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

Compile the final model

In [ ]:
final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the final model

In [ ]:
final_model.fit(final_X_train, final_Y_train, epochs=5, batch_size=32, validation_data=(final_X_validate,final_Y_validate) )



Epoch 1/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8430 - loss: 0.3241 - val_accuracy: 0.9573 - val_loss: 0.1074
Epoch 2/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9868 - loss: 0.0421 - val_accuracy: 0.9621 - val_loss: 0.1059
Epoch 3/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9973 - loss: 0.0094 - val_accuracy: 0.9616 - val_loss: 0.1372
Epoch 4/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9989 - loss: 0.0036 - val_accuracy: 0.9578 - val_loss: 0.2275
Epoch 5/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9991 - loss: 0.0022 - val_accuracy: 0.9595 - val_loss: 0.1941


Test the accuracy of the final model

In [ ]:
final_test_loss, final_test_acc = final_model.evaluate(final_X_test, final_Yl_test)
print(f"Final Test Accuracy: {final_test_acc}")

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9606 - loss: 0.1985
Final Test Accuracy: 0.9593197703361511


predict the validation data set with the final model

In [ ]:
predictions = final_model.predict(final_X_validate)
# print(predictions[:30])  # Display the first 10 predictions

# Convert probabilities to binary predictions
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

# print(binary_predictions[:10])

num_to_print = 10

# Print titles and predictions
print(f"{'Title':<50} {'Prediction':<20}")
print("="*70)

for title, prediction in zip(validation_titles[:num_to_print], binary_predictions[:num_to_print]):
    print(f"{title:<50} {prediction}")

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Title                                              Prediction          
Five Nights at Freddy’s Sequel Delayed for Weird Reason [0]
Why Arizona Sheriff Joe Arpaio’s fate could hang on a single word [1]
Here’s how much you should be tipping your hairdresser [0]
Alan Rickman & Rupert Grint On 'CBGB,' Reuniting Post-'Potter' And Favorite Halloween Costumes [1]
Man swallowed a microSD card and you won't believe what happened next! [0]
Sprite May Be The Best Hangover Cure, Chinese Researchers Say [0]
The Anytime Snack You Won't Feel Guilty About Eating (VIDEO) [1]
You Won’t Believe This Stunning ‘Harry Potter’ Revelation About Professor McGonagall [0]
J.J. Abrams has an answer on if there will be a post-credits scene in the new 'Star Wars' [1]
Kristin Cavallari Talks 'The Hills' Being Fake     [1]


Save model

In [ ]:
# Save the model to a file (HDF5 format)
final_model.save('CNN_clickbait_model_v1.h5')

# Save the model to a directory (SavedModel format)
final_model.save('CNN_clickbait_model_v1.keras')


Download model

In [ ]:

# For HDF5 format
# files.download('my_model.h5')

# For TensorFlow SavedModel format
# Use the directory name instead of a file name
!zip -r CNN_clickbait_model_v1.zip CNN_clickbait_model_v1.keras
from google.colab import files

files.download('CNN_clickbait_model_v1.zip')


  adding: CNN_clickbait_model_v1.keras (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>